<a href="https://colab.research.google.com/github/yihaozhong/479_data_management/blob/main/Pandas_Part_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### More string functions

* upper and lower
* replace 

In [ ]:
# a small toy DataFrame
import pandas as pd
import numpy as np 
df=pd.DataFrame([['A.','c.'],['B.','d.']])
df


,0,1
0,A.,c.
1,B.,d.


In [ ]:
# convert first column to lower case
df[0]=df[0].str.lower()
df

,0,1
0,a.,c.
1,b.,d.


In [ ]:
# chained operations:
# convert second column to upper case
# and then replace the period with an exclamation point
df[1]=df[1].str.upper().str.replace('.','!')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.


,0,1
0,a.,C!
1,b.,D!


## str.split again

In [ ]:
df = pd.DataFrame([['BAZCO', 'https://baz.edu' ],
                   ['Foo Inc', 'http://foo.com']],
              columns=['Name', 'URL'])
df              

,Name,URL
0,BAZCO,https://baz.edu
1,Foo Inc,http://foo.com


Expand option separates into DataFrame columns (otherwise you get a list)

In [ ]:
# expand into separate columns
tmp = df['URL'].str.split('://', expand=True)
# keep as lists in a series
#tmp = df['URL'].str.split('://')
tmp

,0,1
0,https,baz.edu
1,http,foo.com


Now we can add the new columns to the original dataframe

In [ ]:
# add back to original DataFrame
df[['protocol','domain']]=tmp
df

,Name,URL,protocol,domain
0,BAZCO,https://baz.edu,https,baz.edu
1,Foo Inc,http://foo.com,http,foo.com


### DataFrame Global Replace

In [ ]:
# here is a another toy dataframe
df = pd.DataFrame([['foo', -1],
                   ['foo', 12],
                   ['bar', 3]], columns=['a', 'b'])
df                   

,a,b
0,foo,-1
1,foo,12
2,bar,3


Replace all of the instances of "foo" with "qux" anywhere in the dataframe. And replace all of the instances of -1 with 100.

In [ ]:
df=df.replace('foo','qux')
df=df.replace(-1,100)
df

,a,b
0,qux,100
1,qux,12
2,bar,3


## Dealing with Missing Data

In [ ]:
# another toy dataframe
import pandas as pd
df = pd.DataFrame(np.arange(12).reshape((3, 4)),  columns=list('abcd'))
# make two of the values 
df.loc[1, 'd'] = np.nan
df.loc[2, 'c'] = np.nan
df

,a,b,c,d
0,0,1,2.0,3.0
1,4,5,6.0,NaN
2,8,9,NaN,11.0


In [ ]:
df.dtypes

a      int64
b      int64
c    float64
d    float64
dtype: object

Less than ideal: to ignore columns containing missing data

In [ ]:
# one approach: ignoring the columns with missing data
df.dropna(axis=1)

,a,b
0,0,1
1,4,5
2,8,9


You could also drop the rows containing missing data. Again, not ideal.

In [ ]:
df.dropna(axis=0)

,a,b,c,d
0,0,1,2.0,3.0


## Imputation

In statistics, the process of filling in missing data, and of dealing with missing data in general, is called "imputation." This is a very complex topic and whole courses and [books](https://stefvanbuuren.name/fimd/) are on the topic.

The simplest imputation approach is to replace each missing value with the column's mean.

In [ ]:
df

,a,b,c,d
0,0,1,2.0,3.0
1,4,5,6.0,NaN
2,8,9,NaN,11.0


In [ ]:
# mean imputation
# use a dictionary if you want to do it on a per-column basis
# which is usually what you want
# since each column is usually a different measurement like 
# height or weight
df.fillna({'c':df['c'].mean(),'d':df['d'].mean()})

,a,b,c,d
0,0,1,2.0,3.0
1,4,5,6.0,7.0
2,8,9,4.0,11.0


In [ ]:
df

,a,b,c,d
0,0,1,2.0,3.0
1,4,5,6.0,NaN
2,8,9,NaN,11.0


["Hot deck"](https://pubmed.ncbi.nlm.nih.gov/21743766/) imputation just fills missing values with the previous non-missing value in the column. This can be reasonable if you expect similar observations (that is, rows) to be near one another in the list or if you have sorted them so that this is the case.

In [ ]:
df.fillna(method='ffill')

,a,b,c,d
0,0,1,2.0,3.0
1,4,5,6.0,3.0
2,8,9,6.0,11.0


### Binning Values

Perhaps you have a lot of continuous data. It may be useful to categorize that data into bins for easier reporting

Use pd.cut(data,bins,labels=labels)

Where the bins are the boundaries of the bins as a list.

labels can be used to specify text labels for each bin.

Creates a categories object which can be counted with value_counts

In [ ]:
import numpy as np
scores=np.random.randint(60,101,size=100)
# represents 60-69, 70-79, 80-89, 90-100
# upper is inclusive, lower is exclusive
bins=[59,69,79,89,100]
gradelabels=['D','C','B','A' ]
print(scores)
grades=pd.cut(scores,bins, labels=gradelabels)
grades


[ 93  86  76  96  87  94  62  86  81  85  71  74  70  89  99 100  75  95
  91  90  82 100  84  61  73  63  63  67  96  63  95  87  86  87  84  86
  84  69  67  74  92  82  90  91  78  61  76  68  88  79  64  64  84  64
  98  81  88  97  71  94  88  80  88  94  93  94  96  81  65  72  95  95
  73  83  63  73  61  82  71  88  64  90  68  79  95  84  78  63  67  67
  68  74  63  68  74  95  79  66  60  85]


['A', 'B', 'C', 'A', 'B', ..., 'A', 'C', 'D', 'D', 'B']
Length: 100
Categories (4, object): ['D' < 'C' < 'B' < 'A']

In [ ]:
pd.value_counts(grades)

B    28
D    26
A    26
C    20
dtype: int64

Ed Exercise

Take the file height_weight.csv which is found in the Google Drive folder and read it into a Pandas DataFrame. 

Using the pandas cut method, bin the heights into the following groups:

less than 60 inches, 60-64, 64-68, 68-72, 72-76, and over 76.

Then use value_counts to find how many heights fall into each bin.


## Renaming Columns and Rows

In [ ]:
df = pd.DataFrame(np.arange(9).reshape((3, 3)),
                 columns = ['a', 'b', 'c'])
df                 

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8


In [ ]:
# make all of the column names uppercase with a transform function
df2=df.rename(columns=str.upper)
df2


,A,B,C
0,0,1,2
1,3,4,5
2,6,7,8


In [ ]:
# or change individual names with a dictionary
df2=df.rename(columns={'a':'x','c':'z'})
df2

,x,b,z
0,0,1,2
1,3,4,5
2,6,7,8


Changing the column and index names in place by resetting the associated attributes:

In [ ]:
df

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8


In [ ]:
# mapping a function across the columns
df.columns=df.columns.map(str.upper)
df

,A,B,C
0,0,1,2
1,3,4,5
2,6,7,8


In [ ]:
# reset row names with a list
df.index=[5,7,9]
df

,A,B,C
5,0,1,2
7,3,4,5
9,6,7,8


## Constraining values

In [ ]:
df = pd.DataFrame([[2, 3], [1, 50], [20, 4], [3, 45]])
df

,0,1
0,2,3
1,1,50
2,20,4
3,3,45


In [ ]:
(df>40)

,0,1
0,False,False
1,False,True
2,False,False
3,False,True


In [ ]:
(df>40).any()

0    False
1     True
dtype: bool

In [ ]:
(df>40).any(1)

0    False
1     True
2    False
3     True
dtype: bool

In [ ]:
# find all rows with a value over 40
df[(df > 40).any(1)]

,0,1
1,1,50
3,3,45


In [ ]:
# find all rows w/ values in column numbered 1 that's > 10
df[df[1] >  10]

,0,1
1,1,50
3,3,45


In [ ]:
df

,0,1
0,2,3
1,1,50
2,20,4
3,3,45


In [ ]:
# replace those values with 10
df[1][df[1] >  10] = 10
df

,0,1
0,2,3
1,1,10
2,20,4
3,3,10


## Type Conversion

To convert columns from one type to another, use:

* astype(newType)
* pd.to_numeric(colName)
* pd.to_datetime(colName)

The last two allow ignoring type conversion errors.

In [ ]:
data = [['2009', '$500'],
        ['2010', '$1,234'],
        ['2011', 'bad data'],
        ['2012', '$2,507']]
df = pd.DataFrame(data , columns=['date', 'total'])
df

,date,total
0,2009,$500
1,2010,"$1,234"
2,2011,bad data
3,2012,"$2,507"


In [ ]:
# remove dollar digns, remove commaa from total column
t = df['total'].str.replace('$', '').str.replace(',', '')
# convert to numeric, ignoring errors
df['total'] = pd.to_numeric(t, errors='coerce')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,date,total
0,2009,500.0
1,2010,1234.0
2,2011,NaN
3,2012,2507.0


## Converting Strings to Datetime Objects


In [ ]:
s = pd.Series(['Jan 7, 2014', 'May 29, 1993'])
s= pd.to_datetime(s)
s

0   2014-01-07
1   1993-05-29
dtype: datetime64[ns]

## Now can use accessor functions:

In [ ]:
s.dt.month

0    1
1    5
dtype: int64

In [ ]:
s.dt.month_name()

0    January
1        May
dtype: object

In [ ]:
s.dt.weekofyear

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


0     2
1    21
dtype: int64

In [ ]:
s.dt.dayofyear

0      7
1    149
dtype: int64

## Joining Dataframes

pd.merge joins data from one DataFrame with another based on common column values. The default result is the intersection of rows with matching column values, with all columns merged:



In [ ]:
a = pd.DataFrame([[2, 20], [4, 40], [6, 60], [8, 80]],
                 columns=['k', 'col1'])
b = pd.DataFrame([[4, 2], [4, 3], [8, 7]], 
                 columns=['k', 'col2'])
a                

,k,col1
0,2,20
1,4,40
2,6,60
3,8,80


Inner join is the default. That computes an intersection.

In [ ]:
b

,k,col2
0,4,2
1,4,3
2,8,7


In [ ]:
# merge: rows without a matching key are omitted
pd.merge(a,b,on='k')

,k,col1,col2
0,4,40,2
1,4,40,3
2,8,80,7


In [ ]:
# or you can do it this way
a.merge(b,on='k')

,k,col1,col2
0,4,40,2
1,4,40,3
2,8,80,7


In [ ]:
# or this way
b.merge(a,on='k')

,k,col2,col1
0,4,2,40
1,4,3,40
2,8,7,80


## Types of Merges (or Joins)

* 'left': include all keys from the first (left) DataFrame, even if they don't exist in second
* 'right': include all keys from the second (right) DataFrame, even if they don't exist in first
* 'outer': union of keys from both DataFrames
* 'inner': intersection of keys from both DataFrames (default)

Outer Join or Marge: Keeps Everything (Union)


In [ ]:
pd.merge(a, b, on='k', how='outer')

,k,col1,col2
0,2,20,NaN
1,4,40,2.0
2,4,40,3.0
3,6,60,NaN
4,8,80,7.0


Left join or merge

In [ ]:
pd.merge(a,b,on='k', how='left')

,k,col1,col2
0,2,20,NaN
1,4,40,2.0
2,4,40,3.0
3,6,60,NaN
4,8,80,7.0


In [ ]:
pd.merge(a,b,on='k',how='right')

,k,col1,col2
0,4,40,2
1,4,40,3
2,8,80,7


Ed Exercise

Given the following two DataFrames taken from Stack Overflow, do inner, outer, left, and right joins of them on the column "key" using the pandas merge method. Inner join gives only the rows with keys in common, outer join gives all rows with keys in either dataframe, left join gives the rows with keys in the left dataframe, and right join gives the rows with keys in the right dataframe.

(Dataframes given on Ed.) 

In [ ]:
# cartesian product, no key
# combines all pairs of rows
# duplicates columns with same names
pd.merge(a,b,how='cross')

,k_x,col1,k_y,col2
0,2,20,4,2
1,2,20,4,3
2,2,20,8,7
3,4,40,4,2
4,4,40,4,3
5,4,40,8,7
6,6,60,4,2
7,6,60,4,3
8,6,60,8,7
9,8,80,4,2


Ed Exercise

The pandas merge method, when called with the how="cross" option, creates the Cartesian product of two dataframes, which combines each row in the first dataframe with each row in the second, without any key for matching (that is, everything matches.)

Create a dataframe with one column called restaurant_name from the following list: ["Amy's Pizza", "Beth's Pizza", "Charlie's Pizza"].

Create another dataframe with one column called slice_type from the following list: ["plain", "mushroom", "anchovy"].

Then create a dataframe that is the Cartesian product of them, which should have nine rows. Create a new "price" column. Set the prices at Amy's as \$1 for plain, \$1.25 for mushroom, and \$1.50 for anchovy. Then set all of the prices at Beth's at \$1 more than Amy's. Then set the prices at Charlie's at \$1 more than Beth's. Print the resultant dataframe.

## Stacking Dataframes with pd.concat



In [ ]:
d1 = pd.DataFrame(np.arange(9).reshape((3, 3)),
                columns=list('abc'))
d2 = pd.DataFrame(np.arange(10, 19).reshape((3, 3)),
                columns=list('abc'))
d1                

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8


In [ ]:
d2

,a,b,c
0,10,11,12
1,13,14,15
2,16,17,18


pd.concat yields the rows of the second dataframe added at the end of the first dataframe

In [ ]:
d3=pd.concat([d1,d2])
d3

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8
0,10,11,12
1,13,14,15
2,16,17,18


In [ ]:
# might want to update the index
d3.indexO=range(6)
d3

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8
3,10,11,12
4,13,14,15
5,16,17,18


Aggregation

In [ ]:
cols = ["Year", "State", "Title", "Employment", "Salary"]

data = [[2016, "CA", "Web Dev", 22650, 82930],
        [2016, "CA", "DB Admin", 12370, 93960],
        [2016, "NY", "Web Dev", 11410, 81140],
        [2016, "NY", "DB Admin", 6650, 91720],
        [2017, "CA", "Web Dev", 21150, 84270],
        [2017, "CA", "DB Admin", 12030, 95630],
        [2017, "NY", "Web Dev", 11900, 82360],
        [2017, "NY", "DB Admin", 7170, 94330],
        [2018, "CA", "Web Dev", 20170, 86160],
        [2018, "CA", "DB Admin", 10970, 100890],
        [2018, "NY", "Web Dev", 12030, 79880],
        [2018, "NY", "DB Admin", 7100, 99000]]

df = pd.DataFrame(data, columns=cols)
df

,Year,State,Title,Employment,Salary
0,2016,CA,Web Dev,22650,82930
1,2016,CA,DB Admin,12370,93960
2,2016,NY,Web Dev,11410,81140
3,2016,NY,DB Admin,6650,91720
4,2017,CA,Web Dev,21150,84270
5,2017,CA,DB Admin,12030,95630
6,2017,NY,Web Dev,11900,82360
7,2017,NY,DB Admin,7170,94330
8,2018,CA,Web Dev,20170,86160
9,2018,CA,DB Admin,10970,100890


In [ ]:
# set index to one of the columns
tmp=df.set_index('Year')
tmp

,State,Title,Employment,Salary
Year,,,,
2016,CA,Web Dev,22650,82930
2016,CA,DB Admin,12370,93960
2016,NY,Web Dev,11410,81140
2016,NY,DB Admin,6650,91720
2017,CA,Web Dev,21150,84270
2017,CA,DB Admin,12030,95630
2017,NY,Web Dev,11900,82360
2017,NY,DB Admin,7170,94330
2018,CA,Web Dev,20170,86160
